# Solvers

## 3rd party solver installation

Through COBRApy, StrainDesign is already shipped with the free **GLPK** linear programming solver. Alternatively, the more powerful commercial solvers [**IBM CPLEX**](https://www.ibm.com/de-de/products/ilog-cplex-optimization-studio) and [**Gurobi**](https://www.gurobi.com/) can be used by both COBRApy and StrainDesign, and the free solver [**SCIP**](https://scipopt.org/) can be used by StrainDesign. Using one of the GLPK alternatives is preferred, in particular, when using strain design algorithms like MCS, OptKnock etc. since their support of indicator contstraints renders computations significantly more stable.

In the following, you will find installation instructions for the individual solvers.

<div class="alert alert-warning">

**Warning:**

The **free community versions** of CPLEX and Gurobi can be used. However, with larger problems (100+ reactions) their problem size limitations may result in uncaught errors.

</div>


### CPLEX

Together with Gurobi, CPLEX is the perfect choice for computing strain designs. Its stability and support of advanced features like indicator constraints and populating solution pools make it indispensible for genome-scale computations.

<div class="alert alert-info">

**Note:**

You will need an academic or commercial licence to be able to use CPLEX.

</div>

<div class="alert alert-warning">

**Warning:**

**CPLEX is not available for Python 3.13 and above.** The newest CPLEX release (22.1.2) supports Python up to 3.12. If you are using Python 3.13+, please use Gurobi or SCIP instead.

</div>

Download and install the CPLEX suite to a location with non-root access since python will need to build some things to set up the CPLEX-API, later. Make sure that your CPLEX and Python versions are compatible. Currently, CPLEX is not officially available for Python 3.13 and above (the latest release 22.1.2 supports up to Python 3.12). Once the installation is completed, you may use the installation to set up the CPLEX-API with your Python/conda environment.

This can be done either with pip

```
pip install yourCPLEXhome/python/VERSION/PLATFORM
```

or with conda. For an installation with conda, make sure to activate the same Python/conda environment where `cobra` and `straindesign` are installed. Then call 

```
python yourCPLEXhome/python/VERSION/PLATFORM/setup.py install
```

Now CPLEX should be available for your computations. If you face difficulties with building CPLEX, consider downgrading the setuptools package to setuptools==58.2.0.

The official instructions can be found here: https://www.ibm.com/docs/en/icos/22.1.0?topic=cplex-setting-up-python-api

### Gurobi

Similar to CPLEX, Gurobi offers a fast MILP solvers with the advanced features of indicator constraints and solution pooling. The installation steps are similar to the ones of CPLEX.

<div class="alert alert-info">

**Note:**

You will need an academic or commercial license and install the Gurobi solver software.

</div>

Ensure that the versions of Gurobi and Python versions are compatible, install Gurobi on your system and activate your license following the steps from the Gurobi manual. In the next step you will link your Gurobi installation to your Python/conda environment.

Using the command line, navigate to your CPLEX installation path and into the Python folder. The path should look similar to 

```
C:/gurobi950/windows64
```

Make sure to activate the same Python/conda environment where `cobra` and `straindesign` are installed. Then call 

```
python setup.py install
```

If your `gurobipy` package does not work right away, additionally install the gurobi package from conda or PyPi via

```
conda install -c gurobi gurobi
```

or

```
python -m pip install gurobipy
```

Now Gurobi is available for your computations.

The official instructions can be found here: https://support.gurobi.com/hc/en-us/articles/360044290292-How-do-I-install-Gurobi-for-Python-

### SCIP

Less powerful than CPLEX and Gurobi, the open source solver SCIP still offers the solution of MILPs with indicator constraints, which gives it an edge above GLPK in terms of stability. If you want to use SCIP, you may install it via conda or pip:

```
conda install -c conda-forge pyscipopt
```

or

```
python -m pip install pyscipopt
```

<div class="alert alert-warning">

**Warning:**

If you encounter program crashes with SCIP (a dependency of pyscipopt), make sure you use a SCIP version of 8.0.0 or older since newer versions are unreliable in solving MILPs and can produce errors (as of December 2023), this issue might get fixed in the future. 
You can, manually install pyscipopt 4.2.0 and scip version 8.0.0 through ``conda install -c conda-forge pyscipopt=4.2.0 scip=8.0.0``. Keep in mind that SCIP 4.2.0 is currently (December 2023) not available for python>3.10.

</div>

Official website: https://github.com/scipopt/PySCIPOpt

## Solver selection

For any type of LP or MILP-based analysis or design method, four different sovers are supported: **GLPK** (which is built into COBRApy/optlang), **IBM CPLEX**, **Gurobi** and **SCIP**. You can query the available solvers by accessing the set ``straindesign.avail_solvers``. For the subsequent steps we also import the cobra module and load the *E. coli* core model.

In [1]:
import cobra
import straindesign as sd
model = cobra.io.load_model('e_coli_core')

sd.avail_solvers

Set parameter Username
Academic license - for non-commercial use only - expires 2023-07-20


{'cplex', 'glpk', 'gurobi', 'scip'}

You may enforce the use of a specific solver by specifying the "solver"-keyword. E.g., to enforce the use of GLPK, use: 

In [2]:
solution = sd.fba(model, solver='glpk')

print(f"Maximum growth: {solution.objective_value}.")

Maximum growth: 0.873921506969.


By default, the automatic solver selection uses COBRApy's selection. Therefore, StrainDesign will try to use the model's selected solver:

In [3]:
print(f"When the model\'s solver is \'{model.solver.configuration}', StrainDesign selects {sd.select_solver(None,model)}.")

When the model's solver is '<optlang.gurobi_interface.Configuration object at 0x0000014FE00DA400>', StrainDesign selects gurobi.


Otherwise COBRApy's global configuration is used.

In [4]:
print(f"COBRApy\'s solver is \'{cobra.Configuration().solver.__name__}', StrainDesign selects {sd.select_solver()}.")

COBRApy's solver is 'optlang.gurobi_interface', StrainDesign selects gurobi.


In [5]:
model.solver = 'cplex'
sd.select_solver('glpk',model)

'glpk'